In [2]:
from torch import nn
import torch


In [38]:
logits = torch.Tensor([90.0,-10.0,90])
logits


tensor([ 90., -10.,  90.])

In [39]:
labels = torch.tensor([1.0, 0,1])

In [40]:
pred = logits.sigmoid()

In [41]:
bce_loss = nn.BCELoss()
bce_loss(pred,labels)

tensor(1.5133e-05)

In [32]:
pred

tensor([1.0000, 0.2689, 1.0000])

In [33]:
bce_loss = nn.BCEWithLogitsLoss()
bce_loss(logits,labels)

tensor(0.1044)

In [42]:
from datasets import load_dataset

In [43]:
dataset = load_dataset('knowledgator/events_classification_biotech') 

/Users/brewc/PycharmProjects/case_hold_wrong_answers/.venv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for knowledgator/events_classification_biotech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/knowledgator/events_classification_biotech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [45]:
dataset['train'][0]


{'title': "Sarah Polley's Book Recommendations",
 'content': 'Drive Your Plow Over the Bones of The Dead\nby Olga Tokarczuk. I am an incredibly slow reader, but the tone and specificity of the world she creates in this book was something I couldnt leave behind until it was done. Also: All We Sawby Anne Michaels, Fight Nightby Miriam Toews, and The Summer Before the Darkby Doris Lessing.\nId like turned into a Netflix show:\nby Amia Srinivasan. One of the most brain-shattering books Ive ever read. Her thinking is so electrically rigorous and fearless. (I double DARE them to make this into a Netflix show!)\n...I last bought:\n. I rediscovered her poetry lately, and I feel like I dont want to read anything else for a while. She owns desire and submerged things.\n...has the greatest ending:\nby J.D. Salinger. The last page always leaves me breathless. The intimacy and truth of that final page is so arresting and almost painful to read.\nshould be on every college syllabus:\nby Anton Piatig